# Read in Data

In [1]:
import pandas as pd

all_batter_data = pd.read_csv('data/Batting(2023-1980).csv', sep=',')

selected_columns = all_batter_data[['playerID', 'Name', 'teamID', 'yearID']]

selected_columns = selected_columns[selected_columns['teamID'] != '2TM'].dropna()
selected_columns = selected_columns[selected_columns['teamID'] != '3TM'].dropna()
selected_columns = selected_columns[selected_columns['teamID'] != '4TM'].dropna()
selected_columns = selected_columns[selected_columns['teamID'] != '5TM'].dropna()



unique_player_data = selected_columns.drop_duplicates(subset='playerID')


# Set Up Contract Scapper

In [2]:
def create_team_dict():
    dict = {}
    #NL West
    dict['ARI'] = 'arizona-diamondbacks'
    dict['COL'] = 'colorado-rockies'
    dict['LAD'] = 'los-angeles-dodgers'
    dict['SDP'] = 'san-diego-padres'
    dict['SFG'] = 'san-francisco-giants'
    #NL Central
    dict['CHC'] = 'chicago-cubs'
    dict['CIN'] = 'cincinnati-reds'
    dict['MIL'] = 'milwaukee-brewers'
    dict['PIT'] = 'pittsburgh-pirates'
    dict['STL'] = 'st-louis-cardinals'
    #NL East
    dict['ATL'] = 'atlanta-braves'
    dict['MIA'] = 'miami-marlins'
    dict['FLA'] = 'florida-marlins'
    dict['NYM'] = 'new-york-mets'
    dict['PHI'] = 'philadelphia-phillies'
    dict['WSN'] = 'washington-nationals'
    dict['MON'] = 'montreal-expos'
    #AL West
    dict['HOU'] = 'houston-astros'
    dict['LAA'] = 'los-angeles-angels'
    dict['ANA'] = 'anaheim-angels'
    dict['CAL'] = 'california-angels'
    dict['OAK'] = 'oakland-athletics'
    dict['SEA'] = 'seattle-mariners'
    dict['TEX'] = 'texas-rangers'
    #AL Central
    dict['CHW'] = 'chicago-white-sox'
    dict['CLE'] = 'cleveland-indians'
    dict['DET'] = 'detroit-tigers'
    dict['KCR'] = 'kansas-city-royals'
    dict['MIN'] = 'minnesota-twins'
    #AL East
    dict['BAL'] = 'baltimore-orioles'
    dict['BOS'] = 'boston-red-sox'
    dict['NYY'] = 'new-york-yankees'
    dict['TBR'] = 'tampa-bay-rays'
    dict['TBD'] = 'tampa-bay-devil-rays'
    dict['TOR'] = 'toronto-blue-jays'
   
    return dict

team_dict = create_team_dict()


# Get all player names and their teams

In [3]:
# names = selected_columns['Name'].str.rstrip('*#.').drop_duplicates()
# # all_parsed_unique_batter_names = names.dropna().apply(lambda x: x.replace('\xa0', '-')).values
# # all_parsed_unique_batter_names

names = unique_player_data['Name'].str.rstrip('*#.')
names = names.replace("'", "")
names = names.replace(".", "")
unique_player_data.loc[:, 'Name'] = names
unique_player_data

# count = 0
# names_and_teams = {}
# for player in unique_player_data['Name'][0:10]:
#     if type(player) is str:
#         player = player.replace('\xa0', '-')
#         team = unique_player_data[unique_player_data['Name'] == names]['teamID'].values
#         names_and_teams[player] = team
#     print(str(count), ' out of ' + str(len(unique_player_data['Name'])))
#     count += 1 

# names_and_teams



,playerID,Name,teamID,yearID
0,abramcj01,CJ Abrams,WSN,2023.0
1,abreujo02,José Abreu,HOU,2023.0
2,abreuwi02,Wilyer Abreu,BOS,2023.0
3,acunaro01,Ronald Acuna Jr,ATL,2023.0
4,adamewi01,Willy Adames,MIL,2023.0
...,...,...,...,...
51256,weavero01,Roger Weaver,DET,1980.0
51261,wheelga01,Gary Wheelock,SEA,1980.0
51270,wilbote01,Ted Wilborn,NYY,1980.0
51279,wirthal01,Alan Wirth,OAK,1980.0


In [4]:
from unidecode import unidecode

names = unique_player_data['Name'].str.rstrip('*#.').apply(unidecode)
unique_player_data.loc[:, 'Name'] = names

unique_player_data



,playerID,Name,teamID,yearID
0,abramcj01,CJ Abrams,WSN,2023.0
1,abreujo02,Jose Abreu,HOU,2023.0
2,abreuwi02,Wilyer Abreu,BOS,2023.0
3,acunaro01,Ronald Acuna Jr,ATL,2023.0
4,adamewi01,Willy Adames,MIL,2023.0
...,...,...,...,...
51256,weavero01,Roger Weaver,DET,1980.0
51261,wheelga01,Gary Wheelock,SEA,1980.0
51270,wilbote01,Ted Wilborn,NYY,1980.0
51279,wirthal01,Alan Wirth,OAK,1980.0


# Contract Scaper

In [5]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re
import os
import csv

def request_base_html(url):
    response = requests.get(url)
    if response.status_code != 200:
        return None
    
    return response

def get_spotrac_url(player, t):
    domain = 'https://www.spotrac.com/mlb'
    domain_team = domain + '/' + t
    domain_team_player = domain_team + '/' + player + '/transactions/'

    if request_base_html(domain_team_player) is not None:
        return str(domain_team_player)
    
    return None

def make_soup(response):
    return BeautifulSoup(response.text, 'html.parser')

def get_player_info(soup):
    player_info = {}

    for transaction in soup.find_all('div', class_='transitem'):
        year = transaction.find('span', class_='transdate').text.strip()
        year = datetime.strptime(year, '%b %d %Y').year
        info = transaction.find('span', class_='transdesc').text.strip()

        if ('$' in info):
            if year not in player_info:
                player_info[year] = [info]
            else:
                if info not in player_info[year]:
                    player_info[year].append(info)
                
    return player_info

def get_largest_contract(player_info):
    largest_contract = 0
    largest_contract_year = None
    largest_contract_duration = None
    largest_contract_team = None
    for year in player_info:
        for contract in player_info[year]:
            match = re.search(r'\$(\d+(?:,\d+)?)( million)?', contract)
            duration_match = re.search(r'(\d+) year', contract)
            team_match = re.search(r'\((.*?)\)', contract)  # get the text within parentheses
            if match:
                value = float(match.group(1).replace(',', ''))
                if match.group(2):
                    value *= 1e6  # convert to millions if 'million' is present
                if value > largest_contract:
                    largest_contract = value
                    largest_contract_year = year
                    if duration_match:
                        largest_contract_duration = int(duration_match.group(1))
                    if team_match:
                        largest_contract_team = team_match.group(1)
    return largest_contract_year, "${:,.2f}".format(largest_contract), largest_contract_duration, largest_contract_team

def db_contract(player_list, team_dict):
    for player in player_list:
        player_ulr = get_spotrac_url(player, team_dict, 'tran')
        response = request_base_html(player_ulr)
        player_info = get_player_info(make_soup(response))
        largest_contract = get_largest_contract(player_info)

In [6]:
# all_player_spotrac_url = {}
# count = 1
# for player in names_and_teams:

#     all_player_spotrac_url[player] = get_spotrac_url(player, team_dict)

#     print(str(count) + " out of " + str(len(names_and_teams)))
#     count += 1
    
# all_player_spotrac_url





all_player_spotrac_url = {}

count = 1
for player in unique_player_data['Name']:
   

    team = unique_player_data[unique_player_data['Name'] == player]['teamID']

    team = team_dict[team.values[0]]

    domain = 'https://www.spotrac.com/mlb'
    domain_team = domain + '/' + team
    domain_team_player = domain_team + '/' + player.replace(' ', '-') + '/transactions/'

    all_player_spotrac_url[player] = domain_team_player

all_player_spotrac_url



   


{'CJ Abrams': 'https://www.spotrac.com/mlb/washington-nationals/CJ-Abrams/transactions/',
 'Jose Abreu': 'https://www.spotrac.com/mlb/houston-astros/Jose-Abreu/transactions/',
 'Wilyer Abreu': 'https://www.spotrac.com/mlb/boston-red-sox/Wilyer-Abreu/transactions/',
 'Ronald Acuna Jr': 'https://www.spotrac.com/mlb/atlanta-braves/Ronald-Acuna-Jr/transactions/',
 'Willy Adames': 'https://www.spotrac.com/mlb/milwaukee-brewers/Willy-Adames/transactions/',
 'Jordyn Adams': 'https://www.spotrac.com/mlb/los-angeles-angels/Jordyn-Adams/transactions/',
 'Riley Adams': 'https://www.spotrac.com/mlb/washington-nationals/Riley-Adams/transactions/',
 'Ty Adcock': 'https://www.spotrac.com/mlb/seattle-mariners/Ty-Adcock/transactions/',
 'Jo Adell': 'https://www.spotrac.com/mlb/los-angeles-angels/Jo-Adell/transactions/',
 'Ehire Adrianza': 'https://www.spotrac.com/mlb/atlanta-braves/Ehire-Adrianza/transactions/',
 'Jesus Aguilar': 'https://www.spotrac.com/mlb/oakland-athletics/Jesus-Aguilar/transactions

In [7]:
# count = 0
# for player in  all_player_spotrac_url:
#     player_info = get_player_info(make_soup(request_base_html(all_player_spotrac_url[player])))
#     largest_contract = get_largest_contract(player_info)


#     if count == 10:
#         break
#     count +=1



print('here we go: ' + str(len(all_player_spotrac_url)))
print('\n')

count = 1
for name in all_player_spotrac_url:
    try:
        url = all_player_spotrac_url[name]
        player_info = get_player_info(make_soup(request_base_html(url)))
        largest_contract = get_largest_contract(player_info)

        playerID = unique_player_data[unique_player_data['Name'] == name]['playerID'].values[0]
    
        # print('player: ' + name)
        # print('playerID: ' + playerID)
        # print('amount: ' + str(largest_contract[1]))
        # print('length: ' + str(largest_contract[2]))
        # print('year: ' + str(largest_contract[0]))
        # print('team: ' + str(largest_contract[3]))
        # print('pos: ' + 'h')
        # print('\n')

        os.chdir('/Users/cadenparry/Sandbox-MLB-Data')
        filename = 'hitter_contracts.csv'

        # Check if the file does not exist or is empty
        if not os.path.exists(filename) or os.path.getsize(filename) == 0:
            with open(filename, 'a') as file:
                file.write('player,player_id,amount,length,year,team,pos' + '\n')

        # Read existing data
        with open(filename, 'r') as file:
            existing_data = list(csv.reader(file, delimiter=','))

        # Prepare new data
        new_data = [name, str(playerID), str(largest_contract[1]), str(largest_contract[2]), str(largest_contract[0]), str(largest_contract[3]), 'h']

        # Check if new data already exists in the file
        if new_data not in existing_data:
            # If not, append it
            with open(filename, 'a') as file:
                writer = csv.writer(file, delimiter=',')
                writer.writerow(new_data)

                print(name + ' added to the file')
        else:
            print(name + ' already exists in the file')
            
    except:
        print('error with ' + name)
        continue

    print(str(count) + ' out of ' + str(len(all_player_spotrac_url)))
    count += 1

    
 






here we go: 9280


CJ Abrams already exists in the file
1 out of 9280
2 out of 9280
Jose Abreu already exists in the file
3 out of 9280
4 out of 9280
Wilyer Abreu already exists in the file
5 out of 9280
6 out of 9280
Ronald Acuna Jr already exists in the file
7 out of 9280
8 out of 9280
Willy Adames already exists in the file
9 out of 9280
10 out of 9280
Jordyn Adams already exists in the file
11 out of 9280
12 out of 9280
Riley Adams already exists in the file
13 out of 9280
14 out of 9280
Ty Adcock already exists in the file
15 out of 9280
16 out of 9280
Jo Adell already exists in the file
17 out of 9280
18 out of 9280
Ehire Adrianza already exists in the file
19 out of 9280
20 out of 9280
Jesus Aguilar already exists in the file
21 out of 9280
22 out of 9280
Nick Ahmed already exists in the file
23 out of 9280
24 out of 9280
Hanser Alberto already exists in the file
25 out of 9280
26 out of 9280
Ozzie Albies already exists in the file
27 out of 9280
28 out of 9280
Scott Alexander a

Exception ignored in: <finalize object at 0x137ab77c0; dead>
Traceback (most recent call last):
  File "/Users/cadenparry/anaconda3/lib/python3.11/weakref.py", line 585, in __call__
    def __call__(self, _=None):

KeyboardInterrupt: 


Aroldis Chapman already exists in the file
245 out of 9280
246 out of 9280
Matt Chapman already exists in the file
247 out of 9280
248 out of 9280
Michael Chavis already exists in the file
249 out of 9280
250 out of 9280
Jazz Chisholm Jr already exists in the file
251 out of 9280
252 out of 9280
Taylor Clarke already exists in the file
253 out of 9280
254 out of 9280
Kody Clemens already exists in the file
255 out of 9280
256 out of 9280
Ernie Clement already exists in the file
257 out of 9280
258 out of 9280
Oscar Colas already exists in the file
259 out of 9280
260 out of 9280
Dylan Coleman already exists in the file
261 out of 9280
262 out of 9280
Zack Collins already exists in the file
263 out of 9280
264 out of 9280
Michael Conforto already exists in the file
265 out of 9280
266 out of 9280
William Contreras already exists in the file
267 out of 9280
268 out of 9280
Willson Contreras already exists in the file
269 out of 9280
270 out of 9280
Franchy Cordero already exists in the f